本笔记本为[教科书](http://manipulation.csail.mit.edu/pick.html)提供了配套示例。建议将两个窗口并排打开！

In [1]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,  # 添加多体系统和场景图
    DiagramBuilder,              # 系统图构建器
    MeshcatVisualizer,           # Meshcat 可视化工具
    Parser,                      # 解析器
    RigidTransform,              # 刚体变换
    RotationMatrix,              # 旋转矩阵
    StartMeshcat,                # 启动 Meshcat
)

from manipulation.scenarios import SetColor  # 设置颜色工具

In [2]:
# 启动可视化工具。
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


# 计算抓取和预抓取位姿

这里是一个带有浮动 Schunk 夹爪和泡沫砖的简单示例。它按照讲义中的描述定义了抓取位姿，并将其渲染到 3D 可视化工具中。

**自我检查**：尝试将抓取位姿更改为预抓取位姿。你喜欢我在文本中选的这些数值吗？

In [3]:
def grasp_poses_example():
    builder = DiagramBuilder()  # 创建系统图构建器

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)  # 添加多体系统和场景图
    parser = Parser(plant, scene_graph)  # 创建解析器
    parser.SetAutoRenaming(True)  # 自动重命名模型
    grasp = parser.AddModelsFromUrl(
        "package://drake_models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"
    )[0]  # 加载夹爪模型（抓取）
    pregrasp = parser.AddModelsFromUrl(
        "package://drake_models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"
    )[0]  # 加载夹爪模型（预抓取）
    brick = parser.AddModelsFromUrl(
        "package://drake_models/manipulation_station/061_foam_brick.sdf"
    )[0]  # 加载泡沫砖模型
    plant.Finalize()  # 完成模型构建

    B_O = plant.GetBodyByName("base_link", brick)  # 获取砖块的基座
    B_Ggrasp = plant.GetBodyByName("body", grasp)  # 获取夹爪（抓取）主体
    B_Gpregrasp = plant.GetBodyByName("body", pregrasp)  # 获取夹爪（预抓取）主体

    # 将预抓取夹爪设置为绿色且半透明。
    inspector = scene_graph.model_inspector()
    for body_index in plant.GetBodyIndices(pregrasp):
        SetColor(
            scene_graph,
            [0, 0.6, 0, 0.5],  # RGBA 绿色半透明
            plant.get_source_id(),
            inspector.GetGeometries(plant.GetBodyFrameIdOrThrow(body_index)),
        )

    meshcat.Delete()  # 清空可视化
    meshcat.SetProperty("/Background", "visible", False)  # 隐藏背景
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)  # 添加可视化器

    diagram = builder.Build()  # 构建系统图
    context = diagram.CreateDefaultContext()  # 创建默认上下文
    plant_context = plant.GetMyContextFromRoot(context)  # 获取 plant 上下文

    # TODO(russt): 设置物体的随机位姿。
    plant.SetFreeBodyPose(
        plant_context,
        B_O,
        RigidTransform(RotationMatrix.MakeYRotation(np.pi / 6), [0.5, 0, 0.025]),
    )  # 设置物体位姿
    
    # 获取当前物体 O 的位姿
    X_WO = plant.EvalBodyPoseInWorld(plant_context, B_O)

    p_GgraspO = [0, 0.11, 0]  # 抓取点在物体坐标系下的位置
    R_GgraspO = RotationMatrix.MakeXRotation(np.pi / 2.0).multiply(
        RotationMatrix.MakeZRotation(np.pi / 2.0)
    )  # 抓取点的旋转
    X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)  # 抓取点的刚体变换
    X_OGgrasp = X_GgraspO.inverse()  # 逆变换
    X_WGgrasp = X_WO.multiply(X_OGgrasp)  # 世界系下抓取点的位姿

    # 预抓取点在夹爪坐标系下为负 y 方向（见图示！）
    X_GgraspGpregrasp = RigidTransform([0, -0.08, 0])
    X_WGpregrasp = X_WGgrasp @ X_GgraspGpregrasp  # 世界系下预抓取点的位姿

    plant.SetFreeBodyPose(plant_context, B_Ggrasp, X_WGgrasp)  # 设置夹爪（抓取）位姿
    # 同时张开手指。
    plant.GetJointByName("left_finger_sliding_joint", grasp).set_translation(
        plant_context, -0.054
    )
    plant.GetJointByName("right_finger_sliding_joint", grasp).set_translation(
        plant_context, 0.054
    )

    plant.SetFreeBodyPose(plant_context, B_Gpregrasp, X_WGpregrasp)  # 设置夹爪（预抓取）位姿
    # 同时张开手指。
    plant.GetJointByName("left_finger_sliding_joint", pregrasp).set_translation(
        plant_context, -0.054
    )
    plant.GetJointByName("right_finger_sliding_joint", pregrasp).set_translation(
        plant_context, 0.054
    )

    diagram.ForcedPublish(context)  # 强制刷新可视化


grasp_poses_example()  # 运行示例

In [4]:
# （此处可用于补充代码或测试）